In [2]:
!pip install torch torchquantum

  Using cached torch-1.12.1-cp37-cp37m-manylinux1_x86_64.whl (776.3 MB)
  Using cached torchquantum-0.1.0-py2.py3-none-any.whl (33 kB)
  Using cached matplotlib-3.5.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (11.2 MB)
  Using cached torchvision-0.13.1-cp37-cp37m-manylinux1_x86_64.whl (19.1 MB)
  Using cached qiskit-0.37.2-py3-none-any.whl
  Using cached torchpack-0.3.1-py3-none-any.whl (34 kB)
  Using cached tqdm-4.64.0-py2.py3-none-any.whl (78 kB)
  Using cached numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Using cached fonttools-4.37.1-py3-none-any.whl (957 kB)
  Using cached kiwisolver-1.4.4-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.whl (1.1 MB)
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
  Using cached qiskit_aer-0.10.4-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (18.0 MB)
  Using cached qiskit_terra-0.21.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.7 MB)
  Using cached qiskit_ibmq_

In [2]:
!pip install pennylane qiskit pennylane-qiskit

  Using cached PennyLane-0.26.0-py3-none-any.whl (1.0 MB)
  Using cached qiskit-0.38.0-py3-none-any.whl
  Using cached PennyLane_qiskit-0.24.0-py3-none-any.whl (34 kB)
  Using cached retworkx-0.12.0-py3-none-any.whl (10 kB)
  Using cached autograd-1.5-py3-none-any.whl (48 kB)
  Using cached networkx-2.6.3-py3-none-any.whl (1.9 MB)
  Using cached PennyLane_Lightning-0.26.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.7 MB)
  Using cached toml-0.10.2-py2.py3-none-any.whl (16 kB)
  Using cached autoray-0.3.2-py3-none-any.whl (36 kB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl (15 kB)
  Using cached appdirs-1.4.4-py2.py3-none-any.whl (9.6 kB)
  Using cached ninja-1.10.2.4-py2.py3-none-manylinux_2_5_x86_64.manylinux1_x86_64.whl (120 kB)
  Using cached qiskit_terra-0.21.2-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (6.7 MB)
  Using cached qiskit_aer-0.11.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (19.2 MB)
  Using cached qiskit_

In [239]:
from importlib import reload

import datasets
import create_gate_circs
import train_circ

reload(datasets)
reload(create_gate_circs)
reload(train_circ)

<module 'train_circ' from '/root/train_circ.py'>

In [ ]:
from datasets import TorchDataset
from create_gate_circs import get_circ_params, TQCirc, generate_random_gate_circ
from train_circ import train_tq_model

import pickle as pkl
import torch
import os
import numpy as np

num_qubits = 10

device = torch.device('cpu')

dataset = 'mnist_10'

train_data = TorchDataset(dataset, 'angle', 1, True)
test_data = TorchDataset(dataset, 'angle', 1, False)

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=1, sampler=torch.utils.data.RandomSampler(train_data))
test_data_loader = torch.utils.data.DataLoader(train_data, batch_size=1024, sampler=torch.utils.data.RandomSampler(train_data))

val_test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=100, sampler=torch.utils.data.SequentialSampler(test_data))

loss = torch.nn.functional.nll_loss
val_loss = torch.nn.NLLLoss(reduction='none')

for i in range(400, 433):
    curr_dir = './experiment_data/{}/trained_circuits/circ_{}'.format(dataset, i + 1)

    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(curr_dir) 

    model = TQCirc(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, True).to(device)
    opt = torch.optim.SGD(model.parameters(), lr=0.05)
    
    losses_list = []
    accs_list = []
    
    for j in range(1):
        curr_train_dir = curr_dir + '/run_{}'.format(j + 1)

        if os.path.exists(curr_train_dir):
            pass
        else:
            os.mkdir(curr_train_dir)
    
        accs_history = train_tq_model(model, opt, loss, train_data_loader, test_data_loader, 120000, 1000, 1000)
        
#         curr_val_losses = []
#         curr_acc = 0
        
#         for k in range(100):
#             val_x, val_y = next(iter(val_test_data_loader))

#             val_preds = model(val_x)
#             curr_val_losses.append(val_loss(val_preds, val_y.to(torch.long)).detach().numpy().flatten())
        
#             val_class = torch.argmax(val_preds.detach(), 1).detach().numpy().flatten()

#             curr_acc += np.mean(val_class == val_y.detach().numpy().flatten())
        
#         np.save(curr_train_dir + '/accs_history_{}.npy'.format(j + 1), accs_history)
        
#         losses_list.append(curr_val_losses)
#         accs_list.append(curr_acc / 100)
        
#         print(curr_acc)
        
    torch.save(model.state_dict(), curr_dir + '/model.pt')
#     np.save(curr_dir + '/val_losses.npy', losses_list)
#     np.savetxt(curr_dir + '/accs.txt', accs_list)

Step 1 | Accuracy: 0.1015625
Step 1001 | Accuracy: 0.2978515625
Step 2001 | Accuracy: 0.2724609375
Step 3001 | Accuracy: 0.306640625
Step 4001 | Accuracy: 0.365234375
Step 5001 | Accuracy: 0.3779296875
Step 6001 | Accuracy: 0.365234375
Step 7001 | Accuracy: 0.33984375
Step 8001 | Accuracy: 0.3388671875
Step 9001 | Accuracy: 0.3720703125
Step 10001 | Accuracy: 0.3857421875
Step 11001 | Accuracy: 0.375
Step 12001 | Accuracy: 0.4111328125
Step 13001 | Accuracy: 0.3984375
Step 14001 | Accuracy: 0.3623046875
Step 15001 | Accuracy: 0.38671875
Step 16001 | Accuracy: 0.3818359375
Step 17001 | Accuracy: 0.3701171875
Step 18001 | Accuracy: 0.3759765625
Step 19001 | Accuracy: 0.3876953125
Step 20001 | Accuracy: 0.41015625
Step 21001 | Accuracy: 0.3857421875
Step 22001 | Accuracy: 0.3662109375
Step 23001 | Accuracy: 0.3935546875
Step 24001 | Accuracy: 0.39453125
Step 25001 | Accuracy: 0.390625
Step 26001 | Accuracy: 0.4072265625
Step 27001 | Accuracy: 0.390625
Step 28001 | Accuracy: 0.376953125
St

## supernet

In [20]:
def convert_supernet_circ_into_gate_circ(subnet, num_embeds, layer_rots, layer_cnots, num_qubits, angle_embed=False):
    circ_gates = []
    gate_params = []
    weights_bounds = [0]
    inputs_bounds = [0]

    curr_layers = subnet
    num_cnots = len(layer_cnots)
    
    for i in range(1):
        num_qubits = len(layer_rots[0])

        if angle_embed:
            rots = ['ry', 'rx', 'rz']
            
            for j in range(num_embeds):
                circ_gates += [rots[j % 3] for i in range(num_qubits)]
                gate_params += [[i] for i in range(num_qubits)]
                weights_bounds += [0 for i in range(num_qubits)]
                inputs_bounds += [inputs_bounds[-1] + i + 1 for i in range(2 * num_qubits - 1)]            
        else:
            for j in range(num_embeds):
                circ_gates += ['h' for i in range(num_qubits)] + ['ry' for i in range(num_qubits)] + ['cry' for i in range(num_qubits - 1)]
                gate_params += [[i] for i in range(num_qubits)] * 2 + [[i, i + 1] for i in range(num_qubits - 1)]
                weights_bounds += [0 for i in range(3 * num_qubits - 1)]
                inputs_bounds += [inputs_bounds[-1] for i in range(num_qubits)] + [inputs_bounds[-1] + i + 1 for i in range(2 * num_qubits - 1)]

        for j in range(len(curr_layers)):
            circ_gates += layer_rots[curr_layers[j] // num_cnots]
            circ_gates += ['cx' for k in layer_cnots[curr_layers[j] % num_cnots]]

            gate_params += [[k] for k in range(len(layer_rots[curr_layers[j] // num_cnots]))]
            gate_params += layer_cnots[curr_layers[j] % num_cnots]

            weights_bounds += [weights_bounds[-1] + k + 1 for k in range(num_qubits)]
            inputs_bounds += [inputs_bounds[-1] for k in range(num_qubits)]

            weights_bounds += [weights_bounds[-1] for k in range(len(layer_cnots[curr_layers[j] % num_cnots]))]
            inputs_bounds += [inputs_bounds[-1] for k in range(len(layer_cnots[curr_layers[j] % num_cnots]))]
            
    return circ_gates, gate_params, inputs_bounds, weights_bounds

In [ ]:
import pennylane as qml
import numpy as np
import os

from datasets_nt import load_dataset
from create_gate_circs import create_gate_circ
from create_noise_models import noisy_dev_from_backend

dataset = 'fmnist_4'
main_dir = './supernet/fmnist_4/'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'supernet', 1)

num_qubits = 4
num_cnot_configs = 4096
num_circs = 2500
num_embed_layers = 4

device_name = 'ibm_nairobi'
dev = qml.device('lightning.qubit', wires=num_qubits)
# dev = noisy_dev_from_backend(device_name, num_qubits)

inds = np.random.choice(len(x_test), 400, False)
x_test = x_test[inds]
y_test = y_test[inds]

param_nums = [12, 16, 20, 24]

for p in param_nums:
    supernet_dir = main_dir + 'search-{}_params_mb'.format(p)

    num_layers = p // num_qubits

    layer_rots = open(supernet_dir + '/rotations.txt').read().split('\n')[:-1]
    layer_rots = [''.join([j for j in i if j.isupper()]) for i in layer_rots]
    layer_rots = [[i[j * 2:j * 2 + 2].lower() for j in range(len(i) // 2)] for i in layer_rots]

    layer_cnots = open(supernet_dir + '/cnots.txt').read().split('\n')[:-1]
    layer_cnots = [''.join([j for j in i[1:] if j not in ['[', ']', ',', ' ']]) for i in layer_cnots]
    layer_cnots = [[[int(i[2 * j]), int(i[2 * j + 1])] for j in range(len(i) // 2)] for i in layer_cnots]
        
    search_space = len(layer_rots) * len(layer_cnots)
        
    accs = []
    losses = []
    circ_layers = []

    params = np.genfromtxt(supernet_dir + '/training_params.txt')[-1].reshape((num_layers, search_space // num_cnot_configs, num_qubits))
        
#     supernet_device_dir = supernet_dir + '/' + device_name
    supernet_device_dir = supernet_dir
    
#     if not os.path.exists(supernet_device_dir):
#         os.mkdir(supernet_device_dir)
        
    for i in range(num_circs):
        curr_circ_desc = np.random.randint(0, search_space, num_layers)

        curr_params = np.concatenate([params[k, curr_circ_desc[k] // num_cnot_configs] for k in range(num_layers)]).flatten()

        circ_gates, gate_params, inputs_bounds, weights_bounds = convert_supernet_circ_into_gate_circ(curr_circ_desc, num_embed_layers, layer_rots, layer_cnots,
                                                                                                          num_qubits, False) 

        
        circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, [0, 1], 'exp')

        val_exp_list = []
        
        for j in range(len(x_test)):            
            val_exp_list.append(circ(x_test[j], curr_params))
            
        val_exps = np.array(val_exp_list)
        val_losses = np.sum(np.power(val_exps - y_test, 2), 1)
        val_loss = np.mean(val_losses)
        acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)
            
        print(curr_circ_desc, acc, val_loss)

        accs.append(acc)
        losses.append(val_loss)
        circ_layers.append(curr_circ_desc)
        
    np.savetxt(supernet_device_dir + '/searched_circ_layers.txt', np.array(circ_layers))
    np.savetxt(supernet_device_dir + '/searched_circ_accs.txt', accs)
    np.savetxt(supernet_device_dir + '/searched_circ_losses.txt', losses)    

[249049  44034 265397] 0.2425 2.1047132557432207
[ 87226 211152 268499] 0.265 2.0456765414677927
[317287  52713  27456] 0.215 2.159042286032198
[ 93144 133407 273821] 0.2725 2.190506576995243
[ 48056 110540 134615] 0.2125 2.3026549407480603
[212159  96799 247897] 0.265 2.2414444952687362
[ 29999 124639 303074] 0.2225 2.099669748272361
[278052 208793 202272] 0.265 2.105971054269144
[265388  46874   5881] 0.245 2.1016971291620004
[ 37158 257015 153834] 0.23 2.182011042851816
[195380 250955 276474] 0.2725 2.1879655617861307
[260073 211954 188079] 0.25 2.0787463672993027
[ 92613 113289 319156] 0.2475 2.2115480414965294
[ 72652 161442 212549] 0.2425 2.0737649239221274
[139777  98913 262878] 0.235 2.0404842331929616
[ 83758 118094 327003] 0.2275 2.065099655874734
[ 40424 151567 164389] 0.23 2.066795380453275
[318122 298057 325895] 0.265 2.2049522129251744
[178401 186838  49968] 0.2775 2.183891740224077
[226760  88696 125300] 0.235 2.1759904824506977
[137998  68098 186304] 0.2475 2.1398298004

In [ ]:
from datasets_nt import load_dataset
from create_gate_circs import create_gate_circ
from train_circ import mse_vec_loss, train_qnn
from create_noise_models import noisy_dev_from_backend

import pennylane as qml
import numpy as np
import os

dataset = 'fmnist_4'
main_dir = './supernet/fmnist_4/'

x_train, y_train, x_test, y_test = load_dataset(dataset, 'supernet', 1)

num_qubits = 4
num_cnot_configs = 4096
num_circs = 2500
num_embed_layers = 4
meas_qubits = [0, 1]
num_steps = 16000

dev = qml.device('lightning.qubit', wires=num_qubits)

param_nums = [12, 16, 20, 24]
search_nums = [100]
num_trials = [25]

# device_name = 'ibm_nairobi'
# noisy_dev = noisy_dev_from_backend(device_name, num_qubits)

for p in param_nums:
    curr_params_dir =  main_dir + 'search-{}_params_mb'.format(p)

    layer_rots = open(curr_params_dir + '/rotations.txt').read().split('\n')[:-1]
    layer_rots = [''.join([j for j in i if j.isupper()]) for i in layer_rots]
    layer_rots = [[i[j * 2:j * 2 + 2].lower() for j in range(len(i) // 2)] for i in layer_rots]

    layer_cnots = open(curr_params_dir + '/cnots.txt').read().split('\n')[:-1]
    layer_cnots = [''.join([j for j in i[1:] if j not in ['[', ']', ',', ' ']]) for i in layer_cnots]
    layer_cnots = [[[int(i[2 * j]), int(i[2 * j + 1])] for j in range(len(i) // 2)] for i in layer_cnots]
        
#     circ_accs = np.genfromtxt(curr_params_dir + '/{}/searched_circ_accs.txt'.format(device_name))
#     circ_losses = np.genfromtxt(curr_params_dir + '/{}/searched_circ_losses.txt'.format(device_name))
#     circ_layers = np.genfromtxt(curr_params_dir + '/{}/searched_circ_layers.txt'.format(device_name)).astype('int32').reshape((num_circs, p // num_qubits))

    circ_accs = np.genfromtxt(curr_params_dir + '/searched_circ_accs.txt')
    circ_losses = np.genfromtxt(curr_params_dir + '/searched_circ_losses.txt')
    circ_layers = np.genfromtxt(curr_params_dir + '/searched_circ_layers.txt').astype('int32').reshape((num_circs, p // num_qubits))

    for k in range(len(search_nums)):
        search_dir = curr_params_dir + '/search_{}'.format(search_nums[k])

        if not os.path.exists(search_dir):
            os.mkdir(search_dir)

        for i in range(num_trials[k]):
            curr_dir = search_dir + '/trial_{}'.format(i + 1)

            if not os.path.exists(curr_dir):
                os.mkdir(curr_dir)

            chosen_circs = range(search_nums[k] * i, search_nums[k] * i + search_nums[k])
            best_circuit_ind = chosen_circs[np.argmax(circ_accs[chosen_circs])]
            best_subnet = circ_layers[best_circuit_ind]
            print(circ_accs[best_circuit_ind], circ_losses[best_circuit_ind], best_subnet)

            np.savetxt(curr_dir + '/sampled_circuits.txt', circ_layers[chosen_circs])
            np.savetxt(curr_dir + '/sampled_circuit_accs.txt', circ_accs[chosen_circs])

            circ_gates, gate_params, inputs_bounds, weights_bounds = convert_supernet_circ_into_gate_circ(best_subnet, num_embed_layers,
                                                                                                              layer_rots, layer_cnots, num_qubits, False)

            circ = create_gate_circ(dev, circ_gates, gate_params, inputs_bounds,
                                                                weights_bounds, meas_qubits, 'exp', 'adjoint') 
            
#             noisy_circ = create_gate_circ(noisy_dev, circ_gates, gate_params, inputs_bounds,
#                                                                 weights_bounds, meas_qubits, 'exp', None)

            losses_list = []
            accs_list = []
            
            noisy_losses_list = []
            noisy_accs_list = []

            for j in range(5):
                curr_train_dir = curr_dir + '/run_{}'.format(j + 1)

                if os.path.exists(curr_train_dir):
                    pass
                else:
                    os.mkdir(curr_train_dir)


                info = train_qnn(circ, x_train, y_train, x_test, y_test, [weights_bounds[-1]], num_steps, 0.05, 1, mse_vec_loss, verbosity=17300, 
                                                                                                    loss_window=50, init_params=None, 
                                                                                                    acc_thres=1.1, shuffle=True, print_loss=50)
                
                np.savetxt(curr_train_dir + '/params_{}.txt'.format(j + 1), info[-1])
                np.savetxt(curr_train_dir + '/losses_{}.txt'.format(j + 1), info[0])

                val_exps = [circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
                val_loss = np.array([mse_vec_loss(y_test[k], val_exps[k]) for k in range(len(x_test))]).flatten()
                                
#                 noisy_val_exps = [noisy_circ(x_test[i], info[-1][-1]) for i in range(len(x_test))]
#                 noisy_val_loss = np.array([mse_vec_loss(y_test[k], noisy_val_exps[k]) for k in range(len(x_test))]).flatten()

                acc = np.mean(np.sum(np.multiply(val_exps, y_test) > 0, 1) == 2)
#                 noisy_acc = np.mean(np.sum(np.multiply(noisy_val_exps, y_test) > 0, 1) == 2)
#                 acc = np.mean(val_loss < 1)
#                 noisy_acc = np.mean(noisy_val_loss < 1)

                losses_list.append(val_loss)
                accs_list.append(acc)
                
#                 noisy_losses_list.append(noisy_val_loss)
#                 noisy_accs_list.append(noisy_acc)

            np.savetxt(curr_dir + '/accs.txt', accs_list)
            np.savetxt(curr_dir + '/val_losses.txt', losses_list)          
            
#             np.savetxt(curr_dir + '/noisy_accs.txt', noisy_accs_list)
#             np.savetxt(curr_dir + '/noisy_val_losses.txt', noisy_losses_list)  

            np.savetxt(curr_dir + '/gates.txt', circ_gates, fmt="%s")
            np.savetxt(curr_dir + '/gate_params.txt', gate_params, fmt="%s")
            np.savetxt(curr_dir + '/inputs_bounds.txt', inputs_bounds)
            np.savetxt(curr_dir + '/weights_bounds.txt', weights_bounds)
            np.savetxt(curr_dir + '/layers.txt', best_subnet)

0.345 2.009696234236463 [294123 110888 230952]
[2022-10-12 04:11:48.994 tensorflow-2-3-cpu--ml-m5d-2xlarge-8ccc946b8aeb676d8dc85d844a85:24 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-10-12 04:11:49.038 tensorflow-2-3-cpu--ml-m5d-2xlarge-8ccc946b8aeb676d8dc85d844a85:24 INFO profiler_config_parser.py:102] Unable to find config at /opt/ml/input/config/profilerconfig.json. Profiler is disabled.
Step 1 | Sliding Loss Window : 1.1759730776536812
Step 51 | Sliding Loss Window : 1.4265037505704
Step 101 | Sliding Loss Window : 1.4329231399683422
Step 151 | Sliding Loss Window : 1.4616134212739305
Step 201 | Sliding Loss Window : 1.4354738129110292
Step 251 | Sliding Loss Window : 1.438702554207716
Step 301 | Sliding Loss Window : 1.369617941861098
Step 351 | Sliding Loss Window : 1.412582920033048
Step 401 | Sliding Loss Window : 1.3926553066499656
Step 451 | Sliding Loss Window : 1.423180036200992
Step 501 | Sliding Loss Window : 1.4685895807143439
Step 551 | Sliding Loss Wind

## Train correlation circuits

In [ ]:
from datasets import TorchDataset
from create_gate_circs_np import get_circ_params, TQCirc, generate_true_random_gate_circ
from train_circ_np import train_tq_model, TQMseLoss

import pickle as pkl
import torch
import os
import numpy as np

dataset = 'fmnist_2'
curr_dir = './experiment_data/fmnist_2/trained_circuits/'

num_qubits = 4
num_embeds = 16

num_train_steps = 2500
num_test_data = 400

loss = TQMseLoss(1, 4)

dev = qml.device('lightning.qubit', wires=4)
device = 'cpu'

train_data = TorchDataset('fmnist_2', 'angle', 1, reshape_labels=True)
test_data = TorchDataset('fmnist_2', 'angle', 1, False, reshape_labels=True)

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=32, sampler=torch.utils.data.RandomSampler(train_data))
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=32, sampler=torch.utils.data.SequentialSampler(test_data))

for i in range(200):    
    circ_dir = curr_dir + f'circ_{i + 1}'
    
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(circ_dir)

    losses_list = []
    accs_list = []
    
    for j in range(5):
        curr_train_dir = circ_dir + '/run_{}'.format(j + 1)
        
        if os.path.exists(curr_train_dir):
            pass
        else:
            os.mkdir(curr_train_dir)
    
        model = TQCirc(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, False).to(device)
        opt = torch.optim.SGD(model.parameters(), lr=0.05)
    
        curr_loss, curr_acc = train_tq_model(model, 1, opt, loss, train_data_loader, test_data_loader, num_test_data, num_train_steps, 100, 10)
        
        print(curr_loss, curr_acc)
        
        torch.save(model.state_dict(), curr_train_dir + '/model.pt')

        losses_list.append(curr_loss)
        accs_list.append(curr_acc)
        
    np.savetxt(circ_dir + '/val_losses.txt', losses_list)
    np.savetxt(circ_dir + '/accs.txt', accs_list)   

Step 1 | Loss: 1.1663031578063965
Step 101 | Loss: 1.2313156127929688
Step 201 | Loss: 0.8588418364524841
Step 301 | Loss: 0.9828717708587646
Step 401 | Loss: 0.8434625267982483
Step 501 | Loss: 1.056696891784668
Step 601 | Loss: 1.116924524307251
Step 701 | Loss: 1.218027949333191
Step 801 | Loss: 0.9357656240463257
Step 901 | Loss: 0.9779230952262878
Step 1001 | Loss: 1.0684434175491333
Step 1101 | Loss: 0.8001260757446289
Step 1201 | Loss: 0.9930671453475952
Step 1301 | Loss: 0.7227787971496582
Step 1401 | Loss: 0.8548679351806641
Step 1501 | Loss: 0.9094505310058594
Step 1601 | Loss: 0.9162256717681885
Step 1701 | Loss: 0.8881279230117798
Step 1801 | Loss: 1.0617971420288086
Step 1901 | Loss: 1.2626631259918213
Step 2001 | Loss: 1.05287766456604
Step 2101 | Loss: 0.8134095668792725
Step 2201 | Loss: 0.9033404588699341
Step 2301 | Loss: 1.1160088777542114
Step 2401 | Loss: 0.7824491858482361
0.9830600562893037 0.65
Step 1 | Loss: 1.6050665378570557
Step 101 | Loss: 1.197157740592956

In [ ]:
from datasets import TorchDataset
from create_gate_circs_np import get_circ_params, TQCirc, generate_true_random_gate_circ
from train_circ_np import train_tq_model, TQMseLoss

import pickle as pkl
import torch
import os
import numpy as np

dataset = 'vowel_2'
curr_dir = f'./experiment_data/{dataset}/trained_circuits/'

num_qubits = 4
num_embeds = 10

num_train_steps = 1000
num_test_data = 120

num_meas_qubits = 1

loss = TQMseLoss(num_meas_qubits, 4)

dev = qml.device('lightning.qubit', wires=4)
device = 'cpu'

train_data = TorchDataset(dataset, 'angle', 1, reshape_labels=True)
test_data = TorchDataset(dataset, 'angle', 1, False, reshape_labels=True)

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=32, sampler=torch.utils.data.RandomSampler(train_data))
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=32, sampler=torch.utils.data.SequentialSampler(test_data))

for i in range(500, 1000):    
    circ_dir = curr_dir + f'circ_{i + 1}'
    
    circ_gates, gate_params, inputs_bounds, weights_bounds = get_circ_params(circ_dir)

    losses_list = []
    accs_list = []
    
    for j in range(5):
        curr_train_dir = circ_dir + '/run_{}'.format(j + 1)
        
        if os.path.exists(curr_train_dir):
            pass
        else:
            os.mkdir(curr_train_dir)
    
        model = TQCirc(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, False).to(device)
        opt = torch.optim.SGD(model.parameters(), lr=0.05)
    
        curr_loss, curr_acc = train_tq_model(model, num_meas_qubits, opt, loss, train_data_loader, test_data_loader, num_test_data, num_train_steps, 100, 10)
        
        print(curr_loss, curr_acc)
        
        torch.save(model.state_dict(), curr_train_dir + '/model.pt')

        losses_list.append(curr_loss)
        accs_list.append(curr_acc)
        
    np.savetxt(circ_dir + '/val_losses.txt', losses_list)
    np.savetxt(circ_dir + '/accs.txt', accs_list)   

Step 1 | Loss: 0.8239321708679199
Step 101 | Loss: 1.1964417695999146
Step 201 | Loss: 0.7026258707046509
Step 301 | Loss: 1.2873008251190186
Step 401 | Loss: 1.2038551568984985
Step 501 | Loss: 1.116868019104004
Step 601 | Loss: 0.9750521183013916
Step 701 | Loss: 0.9005481600761414
Step 801 | Loss: 1.036630630493164
Step 901 | Loss: 1.0968258380889893
1.1341575537771578 0.6
Step 1 | Loss: 2.1405868530273438
Step 101 | Loss: 0.826111376285553
Step 201 | Loss: 0.9854364395141602
Step 301 | Loss: 1.3741843700408936
Step 401 | Loss: 0.9971245527267456
Step 501 | Loss: 0.981963574886322
Step 601 | Loss: 1.4038256406784058
Step 701 | Loss: 1.123102068901062
Step 801 | Loss: 1.0745538473129272
Step 901 | Loss: 1.0335378646850586
1.1368828878068769 0.5916666666666667
Step 1 | Loss: 1.7253838777542114
Step 101 | Loss: 1.4402258396148682
Step 201 | Loss: 1.3303104639053345
Step 301 | Loss: 0.9967219829559326
Step 401 | Loss: 1.0574920177459717
Step 501 | Loss: 1.0660775899887085
Step 601 | Los

In [175]:
from importlib import reload

import train_circ_np
import datasets
import create_gate_circs_np

reload(train_circ_np)
reload(datasets)
reload(create_gate_circs_np)

<module 'create_gate_circs_np' from '/root/create_gate_circs_np.py'>

## trai human design circs

In [237]:
from datasets import TorchDataset
from datasets_nt import load_dataset
from create_gate_circs_np import get_circ_params, TQCirc, generate_true_random_gate_circ
from train_circ_np import train_tq_model, TQMseLoss
from create_noise_models import noisy_dev_from_backend
from create_human_design_circs import convert_human_design_circ_to_gate_circ

import pickle as pkl
import torch
import os
import numpy as np

embed_method = 'iqp'

dataset = 'mnist_4'
curr_dir = f'./human_design/{embed_method}_basic/{dataset}'

num_qubits = 4
num_embed_layers = 4
num_dataset_reps = 1

num_train_steps = 2500
num_test_data = 2000
num_meas_qubits = 2

param_nums = [24, 28, 32, 36, 40]

if not os.path.exists(curr_dir):
    os.mkdir(curr_dir)

loss = TQMseLoss(num_meas_qubits, 4)
device = 'cpu'

train_data = TorchDataset(dataset, embed_method, num_dataset_reps, reshape_labels=True)
test_data = TorchDataset(dataset, embed_method, num_dataset_reps, False, reshape_labels=True)

train_data_loader = torch.utils.data.DataLoader(train_data, batch_size=32, sampler=torch.utils.data.RandomSampler(train_data))
test_data_loader = torch.utils.data.DataLoader(test_data, batch_size=32, sampler=torch.utils.data.SequentialSampler(test_data))

for param_num in param_nums:
    curr_param_dir = curr_dir + f'/{param_num}_params/'
    
    if not os.path.exists(curr_param_dir):
        os.mkdir(curr_param_dir)
    
    for i in range(1):    
        circ_dir = curr_param_dir

        circ_gates, gate_params, inputs_bounds, weights_bounds = convert_human_design_circ_to_gate_circ(num_qubits, embed_method, 'basic', num_embed_layers,
                                                                                                        param_num // num_qubits)
        
        np.savetxt(curr_param_dir + '/gates.txt', circ_gates, fmt="%s")
        np.savetxt(curr_param_dir + '/gate_params.txt', gate_params, fmt="%s")
        np.savetxt(curr_param_dir + '/inputs_bounds.txt', inputs_bounds)
        np.savetxt(curr_param_dir + '/weights_bounds.txt', weights_bounds)
        
        losses_list = []
        accs_list = []

        for j in range(5):
            curr_train_dir = circ_dir + '/run_{}'.format(j + 1)

            if os.path.exists(curr_train_dir):
                pass
            else:
                os.mkdir(curr_train_dir)

            model = TQCirc(circ_gates, gate_params, inputs_bounds, weights_bounds, num_qubits, False).to(device)
            opt = torch.optim.SGD(model.parameters(), lr=0.05)

            curr_loss, curr_acc = train_tq_model(model, num_meas_qubits, opt, loss, train_data_loader, test_data_loader, num_test_data, num_train_steps, 100, 10)

            print(curr_loss, curr_acc)

            losses_list.append(curr_loss)
            accs_list.append(curr_acc)
            
            torch.save(model.state_dict(), curr_train_dir + '/model.pt')

        np.savetxt(circ_dir + '/val_losses.txt', losses_list)
        np.savetxt(circ_dir + '/accs.txt', accs_list)   

Step 1 | Loss: 2.357640027999878
Step 101 | Loss: 1.7768352031707764
Step 201 | Loss: 1.8842246532440186
Step 301 | Loss: 1.5716921091079712
Step 401 | Loss: 1.653658151626587
Step 501 | Loss: 1.6025004386901855
Step 601 | Loss: 1.3786541223526
Step 701 | Loss: 1.5072001218795776
Step 801 | Loss: 1.5657532215118408
Step 901 | Loss: 1.5411478281021118
Step 1001 | Loss: 1.3236982822418213
Step 1101 | Loss: 1.3589930534362793
Step 1201 | Loss: 1.2263468503952026
Step 1301 | Loss: 1.3140313625335693
Step 1401 | Loss: 1.3451752662658691
Step 1501 | Loss: 1.4813015460968018
Step 1601 | Loss: 1.3610416650772095
Step 1701 | Loss: 1.645634412765503
Step 1801 | Loss: 1.435113549232483
Step 1901 | Loss: 1.311999797821045
Step 2001 | Loss: 1.400325059890747
Step 2101 | Loss: 1.6278877258300781
Step 2201 | Loss: 1.16896653175354
Step 2301 | Loss: 1.5234732627868652
Step 2401 | Loss: 1.3478035926818848
1.4322246648460557 0.5505
Step 1 | Loss: 1.9664428234100342
Step 101 | Loss: 1.7164477109909058
St

In [235]:
from importlib import reload

import datasets_nt
import datasets
import create_gate_circs_np

reload(datasets_nt)
reload(datasets)
reload(create_gate_circs_np)

<module 'create_gate_circs_np' from '/root/create_gate_circs_np.py'>